In [18]:
#import necessory documents.

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense,Dropout,Bidirectional,Embedding
import warnings
warnings.filterwarnings('ignore')

In [6]:
#load data
df = pd.read_excel('../dataset/Restaurant_Reviews.xlsx')

In [8]:
df.columns

Index(['Unnamed: 0', 'Review', 'Liked'], dtype='object')

In [12]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [13]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


### Vocabulary size

In [14]:
#words Vocabulary, we can make it 10,000 to imporove the words representation.
voc_size=5000

In [23]:
messages = df['Review']
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus.append(review)

In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [31]:
print(onehot_repr[0])
print(messages[0])

[2197, 2710, 3595, 192]
Wow... Loved this place.


### Embedding representation.

In [32]:
sent_length=20 #
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [35]:
embedded_docs[0]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 2197, 2710, 3595,  192]])

### Model creation

In [51]:
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(df['Liked'])

In [39]:
X_final.shape

(1000, 20)

In [40]:
y_final.shape

(1000,)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

### Model training

In [52]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 2s 2ms/sample - loss: 0.6924 - acc: 0.5412 - val_loss: 0.6914 - val_acc: 0.5700
Epoch 2/20
800/800 [==============================] - 1s 646us/sample - loss: 0.6835 - acc: 0.7237 - val_loss: 0.6828 - val_acc: 0.5900
Epoch 3/20
800/800 [==============================] - 0s 598us/sample - loss: 0.6376 - acc: 0.6888 - val_loss: 0.6577 - val_acc: 0.6300
Epoch 4/20
800/800 [==============================] - 1s 650us/sample - loss: 0.5208 - acc: 0.7763 - val_loss: 0.5738 - val_acc: 0.7250
Epoch 5/20
800/800 [==============================] - 0s 590us/sample - loss: 0.3832 - acc: 0.8525 - val_loss: 0.5559 - val_acc: 0.7250
Epoch 6/20
800/800 [==============================] - 1s 792us/sample - loss: 0.2487 - acc: 0.9162 - val_loss: 0.6356 - val_acc: 0.7350
Epoch 7/20
800/800 [==============================] - 0s 537us/sample - loss: 0.1691 - acc: 0.9413 - val_loss: 0.7460 - val_acc: 0.7350
Epoc

### Performance Metrics And Accuracy

In [53]:
y_pred=model.predict_classes(X_test)

In [54]:
from sklearn.metrics import confusion_matrix

In [55]:
confusion_matrix(y_test,y_pred)

array([[71, 25],
       [28, 76]], dtype=int64)

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.735

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.72      0.74      0.73        96
           1       0.75      0.73      0.74       104

    accuracy                           0.73       200
   macro avg       0.73      0.74      0.73       200
weighted avg       0.74      0.73      0.74       200

